In [498]:
import pandas as pd
from decimal import Decimal
import json

In [499]:
df = pd.read_csv('total-candidates.csv')

In [500]:
with open('geo/bel.geojson', mode='rt') as f:
    geo_data = json.load(f)

In [501]:
candidates_with_luk = ['lukashenko', 'tihanovkaja', 'against', 'dmitriyev', 'cherechen', 'kanopatskaja', 'corrupted']
trans_candidates = ['Лукашэнка', "Ціханоўская", "супраць усіх", "Дзмітрыеў", "Чэрачань", "Канапацкая", "спорчаны бюлетэнь"]

In [502]:
def get_percentages(record):
    data = df[df['id'].isin(record['properties']['honest_people_ids'])]
    total = data['votes'].sum()

    return {
        c: data[c].sum() / total
        for c in candidates_with_luk
    }


In [503]:
calculated_geo = {
    "type": "FeatureCollection",
    "features": [
        {
            'type': x['type'],
            'geometry': x['geometry'],
            'properties': dict(list({
                'name': x['properties']['name_be'],
                'honest_people_ids': x['properties']['honest_people_ids'],                
            }.items()) + list(get_percentages(x).items())
            ),
        }
        for x in geo_data['features']
    ]
}

In [504]:
LUK_COLOR = 'rgb(0, 0, 255)'
TIH_COLOR = 'rgb(255, 140, 0)'

In [505]:
def compete_style_callback(x):
    data = x['properties']
    
    if (data['lukashenko'] > data['tihanovkaja']) and (data['lukashenko'] > data['against']):
        color = LUK_COLOR
        opacity = data['lukashenko']
    elif (data['tihanovkaja'] > data['lukashenko']) and (data['tihanovkaja'] > data['against']):
        color = TIH_COLOR
        opacity = data['tihanovkaja']
    else:
        color = 'grey'
        opacity = data['against']
        
    return {
        'fillOpacity': opacity,
        'fillColor': color,
    }

In [506]:
from ipyleaflet import GeoJSON, Map

bel_m = Map(center=(53.8, 30), zoom=6)

geo_json = GeoJSON(
    data=calculated_geo,
    style={
        'opacity': 0.8, 'color': 'grey', 'weight': 1
    },
    style_callback=compete_style_callback
)
bel_m.add_layer(geo_json)

bel_m

Map(center=[53.8, 30], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [507]:
def fantasy_style_callback(x):
    data = x['properties']
    
#     luk = data['lukashenko'] + data['kanopatskaja']
#     ag = data['tihanovkaja'] + data['against'] + data['dmitriyev'] + data['cherechen']

    luk = data['lukashenko']
    ag = data['tihanovkaja'] + data['against'] + data['dmitriyev'] + data['cherechen'] + data['kanopatskaja'] + data['corrupted']
    
    if luk > ag:
        color = LUK_COLOR
        opacity = luk
    else:
        color = TIH_COLOR
        opacity = ag
        
    return {
        'fillOpacity': opacity,
        'fillColor': color,
    }

In [508]:
fan_m = Map(center=(53.8, 30), zoom=6)

geo_json = GeoJSON(
    data=calculated_geo,
    style={
        'opacity': 0.8, 'color': 'grey', 'weight': 1
    },
    style_callback=fantasy_style_callback
)
fan_m.add_layer(geo_json)

fan_m

Map(center=[53.8, 30], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [509]:
def tih_style_callback(x):
    data = get_percentages(x)
    
    return {
        'fillOpacity': data['tihanovkaja'] * 1.5,
        'fillColor': TIH_COLOR,
    }

In [510]:
tih_m = Map(center=(53.8, 30), zoom=6)

geo_json = GeoJSON(
    data=geo_data,
    style={
        'opacity': 0.8, 'color': 'grey', 'weight': 1
    },
    style_callback=tih_style_callback
)
tih_m.add_layer(geo_json)

tih_m

Map(center=[53.8, 30], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [511]:
def luk_style_callback(x):
    data = get_percentages(x)
    
    return {
        'fillOpacity': data['lukashenko'],
        'fillColor': LUK_COLOR,
    }

In [512]:
luk_m = Map(center=(53.8, 30), zoom=6)

geo_json = GeoJSON(
    data=geo_data,
    style={
        'opacity': 0.8, 'color': 'grey', 'weight': 1
    },
    style_callback=luk_style_callback
)
luk_m.add_layer(geo_json)

luk_m

Map(center=[53.8, 30], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [513]:
def ag_style_callback(x):
    data = get_percentages(x)
    
    return {
        'fillOpacity': data['against'] * 10,
        'fillColor': 'grey',
    }

In [514]:
ag_m = Map(center=(53.8, 30), zoom=6)

geo_json = GeoJSON(
    data=geo_data,
    style={
        'opacity': 0.8, 'color': 'grey', 'weight': 1
    },
    style_callback=ag_style_callback
)
ag_m.add_layer(geo_json)

ag_m

Map(center=[53.8, 30], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [515]:
def translate_candidate_name(name):
    return dict(zip(candidates_with_luk, trans_candidates))[name]

In [516]:
def get_base_prop(p):
    properties = {
        'Рэгіён': p['name'],
    }
    
    properties.update({
        translate_candidate_name(x): '%.1f %%' % (p[x] * 100)
        for x in candidates_with_luk
    })
    
    return properties

In [517]:
def build_compete_properties(p):
    if p['lukashenko'] > p['tihanovkaja']:
        color = LUK_COLOR
        opacity = p['lukashenko']
    else:
        color = TIH_COLOR
        opacity = p['tihanovkaja']
        
    properties = get_base_prop(p)    
    
    properties['fill'] = color
    properties['fill-opacity'] = opacity
    
    return properties
    

In [518]:
def build_against_luk_properties(p):
    if p['lukashenko'] > 0.5:
        color = LUK_COLOR
        opacity = p['lukashenko']
    else:
        color = TIH_COLOR
        opacity = 1 - p['lukashenko']
    
    properties = get_base_prop(p)
    
    properties['fill'] = color
    properties['fill-opacity'] = opacity
    
    return properties

In [519]:
def build_luk_properties(p):
    properties = get_base_prop(p)
    
    properties['fill'] = LUK_COLOR
    properties['fill-opacity'] = p['lukashenko']
    
    return properties

In [520]:
def build_tih_properties(p):
    properties = get_base_prop(p)
    
    properties['fill'] = TIH_COLOR
    properties['fill-opacity'] = p['tihanovkaja']
    
    return properties

In [521]:
def create_geojson(prop_callback):
    return {
        "type": "FeatureCollection",
        "features": [
            {
                'type': x['type'],
                'geometry': x['geometry'],
                'properties': prop_callback(x['properties']),
            }
            for x in calculated_geo['features']
        ]
    }

In [522]:
compete_geojson = create_geojson(build_compete_properties)

In [523]:
against_luk = create_geojson(build_against_luk_properties)

In [524]:
luk_geo = create_geojson(build_luk_properties)

In [525]:
tih_geo = create_geojson(build_tih_properties)

In [526]:
with open('geo/compete.geojson', mode='wt') as f:
    json.dump(compete_geojson, f)

In [527]:
with open('geo/against-lukashenko.geojson', mode='wt') as f:
    json.dump(against_luk, f)

In [528]:
with open('geo/lukashenko.geojson', mode='wt') as f:
    json.dump(luk_geo, f)

In [529]:
with open('geo/tihanovkaja.geojson', mode='wt') as f:
    json.dump(tih_geo, f)